In [1]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null

# Obtenir les juges d'un cas
Obtenir les décisions json de l'API, retourner tous les juges de cette décision

Mapper les juges à une URL CourtListener

In [4]:
import re
import os
import requests

In [13]:
!wget --header="Authorization: e4a7a06fdd76c060d5323b00bc459f6b69e6265c" "https://case.law/download/bulk_exports/latest/by_jurisdiction/case_text_open/ark/ark_text.zip"
!rm -r "ark_text_20210921"
!unzip ark_text.zip
!xz --decompress ark_text_20210921/data/data.jsonl.xz


--2022-10-25 10:35:55--  https://case.law/download/bulk_exports/latest/by_jurisdiction/case_text_open/ark/ark_text.zip
Resolving case.law (case.law)... 104.18.112.231, 104.18.111.231, 2606:4700::6812:70e7, ...
Connecting to case.law (case.law)|104.18.112.231|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://case.law/download/bulk_exports/20210921/by_jurisdiction/case_text_open/ark/ark_text_20210921.zip [following]
--2022-10-25 10:35:55--  https://case.law/download/bulk_exports/20210921/by_jurisdiction/case_text_open/ark/ark_text_20210921.zip
Reusing existing connection to case.law:443.
HTTP request sent, awaiting response... 200 OK
Length: 161152107 (154M) [application/zip]
Saving to: ‘ark_text.zip’

ark_text.zip        100%[===================>] 153.69M  21.9MB/s    in 7.8s    

2022-10-25 10:36:04 (19.6 MB/s) - ‘ark_text.zip’ saved [161152107/161152107]

rm: cannot remove 'ark_text_20210921': No such file or directory
Archive:  ark_text.zip
 extra

In [23]:
import json
import pandas as pd

def extract_json(json_str):
  text = json.loads(json_str)
  if text["casebody"]["data"]["opinions"]:
    opinion = text["casebody"]["data"]["opinions"][0]["text"]
    court = text["court"]["name"]
    url = text["frontend_url"]
    date = text["decision_date"]
    name = text["name_abbreviation"]
    author = text["casebody"]["data"]["opinions"][0]["author"]
    return (opinion, author, court, url, date, name)

with open(f'/content/ark_text_20210921/data/data.jsonl', 'r') as json_file:
    results = []
    for json_str in list(json_file):
      try:
        results.append(extract_json(json_str))
      except:
        print(f"Could not be parsed : {json_str} ")
    df = pd.DataFrame(results, columns=["text", "author", "court", "url", "date", "name"]).to_csv("ark.csv")

In [24]:
cases = pd.read_csv("ark.csv")
cases

,Unnamed: 0,text,author,court,url,date,name
0,0,"ROBERT J. GLADWIN, Judge\nh The Arkansas Worke...","ROBERT J. GLADWIN, Judge",Arkansas Court of Appeals,https://cite.case.law/sw3d/533/596/,2017-11-08,"Stoker v. Thomas Randal Fowler, Inc."
1,1,"RAYMOND R. ABRAMSON, Judge\n11 Larry L. Harris...","RAYMOND R. ABRAMSON, Judge",Arkansas Court of Appeals,https://cite.case.law/sw3d/533/648/,2017-11-15,"Harrison v. Street & Performance, Inc."
2,2,"RAYMOND R. ABRAMSON, Judge\nOn August 25, 2016...","RAYMOND R. ABRAMSON, Judge",Arkansas Court of Appeals,https://cite.case.law/sw3d/533/130/,2017-11-01,Sanders v. State
3,3,"N. MARK KLAPPENBACH, Judge\n| ¶ Appellant Wayn...","N. MARK KLAPPENBACH, Judge",Arkansas Court of Appeals,https://cite.case.law/sw3d/533/587/,2017-11-01,Ray v. State
4,4,"ROBERT J. GLADWIN, Judge\nliThe Ouachita Count...","ROBERT J. GLADWIN, Judge",Arkansas Court of Appeals,https://cite.case.law/sw3d/533/654/,2017-11-15,Meyers v. Arkansas Department of Human Services
...,...,...,...,...,...,...,...
60577,60577,NaN,NaN,"Court of Appeals of Arkansas, DIVISION IV",https://cite.case.law/sw3d/567/888/,2018-12-12,"Mountain Crest, LLC v. Kimbro"
60578,60578,"LARRY D. VAUGHT, Judge\nTina Webb appeals the ...","LARRY D. VAUGHT, Judge","Court of Appeals of Arkansas, DIVISION II",https://cite.case.law/sw3d/567/86/,2018-12-12,"Webb v. Wal-Mart Assocs., Inc."
60579,60579,"MIKE MURPHY, Judge\nOn October 23, 2017, appel...","MIKE MURPHY, Judge","Court of Appeals of Arkansas, DIVISION II",https://cite.case.law/sw3d/567/80/,2018-12-12,J.L. v. State
60580,60580,"Jim Hannah, Justice.\nThis matter arises from ...","Jim Hannah, Justice.",Arkansas Supreme Court,https://cite.case.law/ark/356/106/,2004-02-12,McDonald v. State


In [28]:
not_useful = ["memorandum", "chief", "circuit","justice", "per curiam","senior", "district", "judge",  "justice","and", "associate","mr", "mrs", "ms", "presiding","opinion","the court", "dissenting"]
courtlistener_people_api = "https://www.courtlistener.com/api/rest/v3/people/"

def get_author_name(name):
    judges = []
    try:
        authors = name.split(', ')
        for author in #TODO:
            original_author = author
            author_name = fix_name(author)
            first_name = ''
            name_parts = author_name.split(' ')
            if len(name_parts) > 1:
                author_name = name_parts[-1]
                if len(name_parts[0]) > 1:
                    first_name = name_parts[0]
            if len(author_name) > 1:
                author_name = author_name.#TODO Capitalize
                if '-' in author_name:
                    # Gardez les césures dans les noms de personnes, mettez une majuscule à chaque partie césurée.
                    name_parts = author_name.split('-')
                    author_name = ''
                    for idx,part in enumerate(name_parts):
                        if idx < len(name_parts) - 1:
                            author_name += part.capitalize() + '-' 
                        else:
                            author_name += part.capitalize()
                url = "%s?name_last=%s" % (courtlistener_people_api, author_name)
                if first_name:
                    url += "&name_first=%s" % first_name.capitalize()
                judges.append([url, original_author, author_name, first_name.capitalize()])
    except:
        # Pour une raison ou une autre, l'opinion n'a pas d'auteur. 
        # Parfois, c'est parce que les juges ont décidé que l'affaire ne doit pas être traitée comme un précédent.
        pass
    return judges

def fix_name(name):
    name = re.sub('[^0-9a-zA-Z\-\s]+', '', name.lower())
    for #En utilisant la liste not_usefull, supprimer les mots inutiles:
        name = name.replace(nu,'')    
    return name.strip()


judges = cases["author"].apply(get_author_name)

# print first five
print(judges[0:5])

0    [[https://www.courtlistener.com/api/rest/v3/pe...
1    [[https://www.courtlistener.com/api/rest/v3/pe...
2    [[https://www.courtlistener.com/api/rest/v3/pe...
3    [[https://www.courtlistener.com/api/rest/v3/pe...
4    [[https://www.courtlistener.com/api/rest/v3/pe...
Name: author, dtype: object


In [34]:
import pandas as pd
judges_2 = [item for sublist in judges for item in sublist]
judge_df = pd.DataFrame(data=judges_2, columns=["best_guess_url","original", "last_name","first_name"])
judge_df.head()

,best_guess_url,original,last_name,first_name
0,https://www.courtlistener.com/api/rest/v3/peop...,ROBERT J. GLADWIN,Gladwin,Robert
1,https://www.courtlistener.com/api/rest/v3/peop...,RAYMOND R. ABRAMSON,Abraon,Raymond
2,https://www.courtlistener.com/api/rest/v3/peop...,RAYMOND R. ABRAMSON,Abraon,Raymond
3,https://www.courtlistener.com/api/rest/v3/peop...,N. MARK KLAPPENBACH,Klappenbach,
4,https://www.courtlistener.com/api/rest/v3/peop...,ROBERT J. GLADWIN,Gladwin,Robert


1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)